In [15]:
import os
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [3]:
# Define the directory where you want to save the files
folder_path = r"C:\Users\idea\Desktop\C'est ma merde\General Assembly - Data Science Immersive\clone\DSIF-SG-12\project_4\assets"

# Ensure the directory exists
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Define headers for the HTTP request
headers = {
    "User-Agent": "PostmanRuntime/7.26.5",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
}

# Function to safely make a GET request
def safe_get(url, headers=None):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request exception occurred: {req_err}")

# Pause the execution for 1 second
time.sleep(1)

# Use the safe_get function to send a GET request
response = safe_get("http://www.weather.gov.sg/climate-historical-daily/")

if response:
    soup = BeautifulSoup(response.text, "html.parser").find("ul", {"class": "dropdown-menu long-dropdown"}).find_all("li")
    cities_and_codes = {
        t.find("a").getText(strip=True): re.search(r'(S\d+)', t.find("a")['onclick']).group(1)
        for t in soup
    }

    # Generator function to yield dates from 2014 to 2018 (inclusive) for each month
    def get_dates():
        yield from (
            [(y, f"0{m}" if m < 10 else m) for y in range(2014, 2019) for m in range(1, 13)]
        )

    # Base URL for the files
    files_url = "http://www.weather.gov.sg/files/dailydata/DAILYDATA_"
    for city, code in cities_and_codes.items():
        for date in get_dates():
            year, month = date
            csv_url = f"{files_url}{code}_{year}{month}.csv"
            response = safe_get(csv_url)
            if response:
                file_path = os.path.join(folder_path, f"{city.replace(' ', '_')}_{csv_url.split('/')[-1]}")
                try:
                    with open(file_path, "wb") as f:
                        f.write(response.content)
                    print(f"Data for {city} for {month}/{year} saved to {file_path}")
                except IOError as e:
                    print(f"Failed to write file: {e}")
            else:
                print(f"No data available for {city} for {month}/{year}...")


Data for Admiralty for 01/2014 saved to C:\Users\idea\Desktop\C'est ma merde\General Assembly - Data Science Immersive\clone\DSIF-SG-12\project_4\assets\Admiralty_DAILYDATA_S104_201401.csv
Data for Admiralty for 02/2014 saved to C:\Users\idea\Desktop\C'est ma merde\General Assembly - Data Science Immersive\clone\DSIF-SG-12\project_4\assets\Admiralty_DAILYDATA_S104_201402.csv
Data for Admiralty for 03/2014 saved to C:\Users\idea\Desktop\C'est ma merde\General Assembly - Data Science Immersive\clone\DSIF-SG-12\project_4\assets\Admiralty_DAILYDATA_S104_201403.csv
Data for Admiralty for 04/2014 saved to C:\Users\idea\Desktop\C'est ma merde\General Assembly - Data Science Immersive\clone\DSIF-SG-12\project_4\assets\Admiralty_DAILYDATA_S104_201404.csv
Data for Admiralty for 05/2014 saved to C:\Users\idea\Desktop\C'est ma merde\General Assembly - Data Science Immersive\clone\DSIF-SG-12\project_4\assets\Admiralty_DAILYDATA_S104_201405.csv
Data for Admiralty for 06/2014 saved to C:\Users\idea\D

In [28]:
# Define the directory where your CSV files are stored
folder_path = "../assets/"

file_list = os.listdir(folder_path)
file_list

['Admiralty_DAILYDATA_S104_201401.csv',
 'Admiralty_DAILYDATA_S104_201402.csv',
 'Admiralty_DAILYDATA_S104_201403.csv',
 'Admiralty_DAILYDATA_S104_201404.csv',
 'Admiralty_DAILYDATA_S104_201405.csv',
 'Admiralty_DAILYDATA_S104_201406.csv',
 'Admiralty_DAILYDATA_S104_201407.csv',
 'Admiralty_DAILYDATA_S104_201408.csv',
 'Admiralty_DAILYDATA_S104_201410.csv',
 'Admiralty_DAILYDATA_S104_201411.csv',
 'Admiralty_DAILYDATA_S104_201412.csv',
 'Admiralty_DAILYDATA_S104_201501.csv',
 'Admiralty_DAILYDATA_S104_201502.csv',
 'Admiralty_DAILYDATA_S104_201503.csv',
 'Admiralty_DAILYDATA_S104_201504.csv',
 'Admiralty_DAILYDATA_S104_201505.csv',
 'Admiralty_DAILYDATA_S104_201506.csv',
 'Admiralty_DAILYDATA_S104_201507.csv',
 'Admiralty_DAILYDATA_S104_201508.csv',
 'Admiralty_DAILYDATA_S104_201509.csv',
 'Admiralty_DAILYDATA_S104_201510.csv',
 'Admiralty_DAILYDATA_S104_201511.csv',
 'Admiralty_DAILYDATA_S104_201512.csv',
 'Admiralty_DAILYDATA_S104_201601.csv',
 'Admiralty_DAILYDATA_S104_201602.csv',


In [29]:
len(file_list)

3530

In [32]:
# Define the directory where your CSV files are stored
folder_path = "../assets/raw_scrape"

# List to hold data from each CSV file
data_frames = []

# Loop through all files in the directory
for file_name in os.listdir(folder_path):
    # Check if the file is a CSV
    if file_name.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        try:
            # Try reading with utf-8 encoding
            df = pd.read_csv(file_path, index_col=None, header=0)
        except UnicodeDecodeError:
            # If utf-8 encoding fails, try 'ISO-8859-1'
            df = pd.read_csv(file_path, index_col=None, header=0, encoding='ISO-8859-1')
        # Append the DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames into one
combined_csv = pd.concat(data_frames, axis=0, ignore_index=True)
combined_csv.head()

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,Admiralty,2014,1,1,0.0,0.0,0.0,0.0,26.3,28.8,24.3,10.8,34.2
1,Admiralty,2014,1,2,0.0,0.0,0.0,0.0,26.9,30.7,24.6,11.5,38.2
2,Admiralty,2014,1,3,4.6,4.4,4.4,4.6,26.5,29.5,24.9,9.7,34.2
3,Admiralty,2014,1,4,3.8,3.8,3.8,3.8,26.4,31.2,24.7,8.1,32.8
4,Admiralty,2014,1,5,3.6,1.2,1.6,2.0,24.6,25.3,23.6,8.4,27.0


In [33]:
len(combined_csv)

107392

In [35]:
# Save the combined CSV to a new file
combined_csv.to_csv("../assets/daily_weather_2014_to_2018.csv", index=False)